<a href="https://colab.research.google.com/github/Pranithaadari/Legal-AI-App/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "ibm-watsonx-ai==0.2.6"
!pip install "langchain==0.1.16"
!pip install "langchain-ibm==0.1.4"
!pip install "huggingface==0.0.1"
!pip install "huggingface-hub==0.21.4"
!pip install "sentence-transformers==2.5.1"
!pip install "chromadb==0.4.24"
!pip install "wget==3.2"
!pip install -U langchain-community


  Using cached langchain-0.1.16-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
Using cached langchain-0.1.16-py3-none-any.whl (817 kB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.16
    Uninstalling langchain-community-0.3.16:
      Successfully uninstalled langchain-community-0.3.16
  Attempting uninstall: langchain
    Found existing insta

In [ ]:
#You can use this section to suppress warnings generated by your code:

def warn(*args, **kwargs):

    pass

import warnings

warnings.warn=warn

warnings.filterwarnings ('ignore')

from langchain_community.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter

from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceEmbeddings

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain.chains import ConversationalRetrievalChain

from langchain.memory import ConversationBufferMemory

from ibm_watsonx_ai.foundation_models import Model

from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods

from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

import wget

In [ ]:
import shutil

# Define the source and destination
filename = 'software-maintenance-agreement.text'
url = '/content/free-software-maintenance-agreement.docx'

# Copy the file from the source to the destination
shutil.copy(url, filename)
print('File downloaded successfully!')

File downloaded successfully!


In [ ]:
with open(filename,'rb') as f:
    data = f.read()
    contents=data.decode('latin-1')
   # print(contents)

In [ ]:
loader=TextLoader(filename,encoding='latin-1')
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
texts=text_splitter.split_documents(documents)
print(len(texts))

2


In [ ]:
embeddings=HuggingFaceEmbeddings()
docsearch=Chroma.from_documents(texts,embeddings)
print('document ingested')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

document ingested


In [ ]:
model_id='google/flan-ul2'

In [ ]:
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 130,
    GenParams.MAX_NEW_TOKENS: 256,
    GenParams.TEMPERATURE: 0.5
}

In [ ]:
# **Make sure to replace these with your actual credentials**
credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"0mHrhwRru3pYpyJLOy73Q00xDVnYaT_dv1weEvDwclNP"
}

project_id="b6ee87bb-8411-4a01-afd7-9dd74aef532b"

In [ ]:
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

In [ ]:
flan_ul2_llm=WatsonxLLM(model=model)

In [ ]:
#qa=RetrievalQA.from_chain_type(llm=flan_ul2_llm,
                             #chain_type="stuff",
                              #retriever= docsearch.as_retriever(),
                             #return_source_documents=False)
#query="Summarize the documents"
#qa.invoke(query)

In [5]:
!pip install python-docx
import docx
import os
import re
import json
from transformers import pipeline
import pandas as pd

def load_and_transformer(file_path):
  if not os.path.exists(file_path):
    raise ValueError(f"File {file_path} does not exist")

  # Use the docx library to read the content of the docx file
  doc = docx.Document(file_path)
  document = '\n'.join([paragraph.text for paragraph in doc.paragraphs])

  chunks=[document[i:i+1000] for i in range(0,len(document),1000)]
  return chunks

#step 2:Risk detection using Hugging Face

def risk_detection(chunks):
  model_name="google/flan-t5-base"
  nlp=pipeline("text2text-generation",model=model_name)

  results=[]
  for chunk in chunks:
    #Define a prompt for risk analysis
    prompt=(
        "Analyze the following text for potential risks,hidden obligations,"
        "or dependencies,and provide recommendations:\n\n"+chunk
    )
    #Generate response
    result=nlp(prompt,max_length=200,do_sample=False)
    recommendation = result[0]['generated_text'].split('.')[-2].strip() + '.' if '.' in result[0]['generated_text'] else result[0]['generated_text']
    results.append({"context":chunk,"analysis":result[0]['generated_text'],"recommendation": recommendation})
  return results

#Step 3:Main execution
def main(file_path):
  print("Loading and preprocessing document...")
  chunks=load_and_transformer(file_path)

  print("Detecting risks and generatind recommendations")
  analysis=risk_detection(chunks)

  #save results to a JSON file
  output_path="risk_analysis.json"
  with open(output_path,"w",encoding="utf-8") as f:
    json.dump(analysis,f,ensure_ascii=False,indent=4)

  print(f"Analysis complete.Results saved to {output_path}")

#Execute the pipeline
if __name__=="__main__":
  file_path="free-software-maintenance-agreement.docx"
  main(file_path) #Call the main function here to create the json

# Now you can safely read from the json file
import json

with open("risk_analysis.json","r",encoding="utf-8") as f:
    data= json.load(f)
# Print each entry's context and analysis
for entry in data:
    print("Context:\n", entry["context"])
    print("Analysis:\n", entry["analysis"])
    print("Recommendation:\n",entry["recommendation"])
    print("-"*50)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 7.9 MB/s eta 0:00:00
Loading and preprocessing document...
Detecting risks and generatind recommendations


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Analysis complete.Results saved to risk_analysis.json
Context:
 Free software maintenance agreement: cover
1.	This template legal document was produced and published by Docular Limited.
2.	We control the copyright in this template, and you may only use this template in accordance with the licensing provisions in our terms and conditions. Those licensing provisions include an obligation to retain the attribution / credit incorporated into the template.
3.	Buy a licence to use this template without the attribution / credit here:
https://www.website-contracts.co.uk/software-maintenance-agreement.html
4.	You will need to edit this template before use. Guidance notes to help you do so are set out at the end of the template. During the editing process, you should delete those guidance notes and this cover sheet. Square brackets in the body of the document indicate areas that require editorial attention. "ORs" in the body of the document indicate alternative provisions. By the end of the edit

In [6]:
import json
import pandas as pd
with open("risk_analysis.json","r",encoding="utf-8") as f:
    data=json.load(f)

df=pd.DataFrame(data)
df

,context,analysis,recommendation
0,Free software maintenance agreement: cover\n1....,This template is intended for use by users of ...,This template is intended for use by users of ...
1,"of the document, and only one alternative fro...",[]] or [][][]] having its registered office at...,[]] or [][][]] having its registered office at...
2,istration number]) having its registered offic...,[the Customer's] time spent performing [the Se...,[the Customer's] time spent performing [the Se...
3,yable amounts as may be agreed in writing by t...,[additional list items],[additional list items]
4,"d ""Released"" shall be construed accordingly);\...","a ""Contract"" between the parties.","a ""Contract"" between the parties."
5,e into force upon the Effective Date.\n3.2\tTh...,"if the Customer is not paid in full, the Provi...","if the Customer is not paid in full, the Provi..."
6,e Provider has given to the Customer at least ...,"Provider terminates the Maintenance Services, ...","Provider terminates the Maintenance Services, ..."
7,arges are based in whole or part upon the time...,[the date of the d]],[the date of the d]]
8,"ate of execution of this Agreement]][, providi...",[d],[d]
9,"ebit card, credit card, direct debit or bank t...",2% per annum above the Bank of England base ra...,2% per annum above the Bank of England base ra...


In [7]:
pip install pandas openpyxl

In [8]:
import openpyxl
output_file = "output.xlsx"
df.to_excel(output_file, index=False, engine='openpyxl')

print(f"Excel file created successfully at {output_file}")

Excel file created successfully at output.xlsx


In [9]:
import json
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

SPREADSHEET_ID = "1hBn9puxi_YUqx4lHZJa-aAp6wqfuw_P5AgCyYPmwkTc"
RANGE_NAME = "Sheet1!A1"

SERVICE_ACCOUNT_FILE ="legal-doc-449015-456b16bcef2f.json"# Path to your credentials JSON file
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# Load credentials and build the service
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build("sheets", "v4", credentials=credentials)

# Prepare the data
values = [df.columns.tolist()] + df.values.tolist()
body = {"values": values}

# Update the Google Sheet
try:
    result = (
        service.spreadsheets()
        .values()
        .update(
            spreadsheetId=SPREADSHEET_ID,
            range=RANGE_NAME,
            valueInputOption="RAW",  # or 'USER_ENTERED' for formatted input
            body=body,
        )
        .execute()
    )
    print(f"{result.get('updatedCells')} cells updated.")
except Exception as e:
    print(f"An error occurred: {e}")
    # This will print more helpful error messages, including the 403 if the API is not enabled.

129 cells updated.


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_gmail_alert(to_email, subject, body):
    """
    Sends an email alert using Gmail's SMTP server.

    Args:
        to_email (str): Recipient's email address.
        subject (str): Email subject.
        body (str): Email content.

    Returns:
        None
    """
    # Gmail credentials
    sender_email = "pranithaprani395@gmail.com"  # Replace with your Gmail address
    sender_password = "cmqq nkps kcoj sfrn"  # Replace with your Gmail App Password
    recipient_email="22071a6765@vnrvjiet.in"

    # Create the email message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    try:
        # Connect to Gmail's SMTP server
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
          server.set_debuglevel(1)
          server.starttls()
          server.login(sender_email,sender_password)
          server.send_message(msg)
          print(f"Email sent successfully to {to_email}!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Example usage
send_gmail_alert("22071a6765@vnrvjiet.in", "Risk Analysis Report", "Please find the attached risk analysis report.")


send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [34.91.142.22]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your service, [34.91.142.22]\nSIZE 35882577\n8BITMIME\nSTARTTLS\nENHANCEDSTATUSCODES\nPIPELINING\nCHUNKING\nSMTPUTF8'
send: 'STARTTLS\r\n'
reply: b'220 2.0.0 Ready to start TLS\r\n'
reply: retcode (220); Msg: b'2.0.0 Ready to start TLS'
send: 'ehlo [172.28.0.12]\r\n'
reply: b'250-smtp.gmail.com at your service, [34.91.142.22]\r\n'
reply: b'250-SIZE 35882577\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH2 PLAIN-CLIENTTOKEN OAUTHBEARER XOAUTH\r\n'
reply: b'250-ENHANCEDSTATUSCODES\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-CHUNKING\r\n'
reply: b'250 SMTPUTF8\r\n'
reply: retcode (250); Msg: b'smtp.gmail.com at your servi

Email sent successfully to 22071a6765@vnrvjiet.in!


reply: b'250 2.0.0 OK  1738057281 4fb4d7f45d1cf-5dc186397c0sm6824234a12.35 - gsmtp\r\n'
reply: retcode (250); Msg: b'2.0.0 OK  1738057281 4fb4d7f45d1cf-5dc186397c0sm6824234a12.35 - gsmtp'
data: (250, b'2.0.0 OK  1738057281 4fb4d7f45d1cf-5dc186397c0sm6824234a12.35 - gsmtp')
send: 'QUIT\r\n'
reply: b'221 2.0.0 closing connection 4fb4d7f45d1cf-5dc186397c0sm6824234a12.35 - gsmtp\r\n'
reply: retcode (221); Msg: b'2.0.0 closing connection 4fb4d7f45d1cf-5dc186397c0sm6824234a12.35 - gsmtp'
